In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import one_hot

import numpy as np
import h5py
import pickle

from sklearn.model_selection import train_test_split
import pandas as pd

import sys
sys.path.append('../common')
from dataset import TMH
from models import CNN
from cait_models import cait_S24_224, cait_tmh

### Create dataloader

In [3]:
dataset = TMH(embeddings_path="/Users/fga/data/tmh/embeddings.h5",
                protein_hashes_path="../seq_anno_hash.pickle",
                train_ids="../data_splits/train_prot_id_labels.csv")
dataloader = DataLoader(dataset, batch_size=4)

test_dataset = TMH(embeddings_path="/Users/fga/data/tmh/embeddings.h5",
                protein_hashes_path="../seq_anno_hash.pickle",
                train_ids="../data_splits/test_prot_id_labels.csv")
test_dataloader = DataLoader(test_dataset, batch_size=4)

/Users/fga/Docs/cls-protein-prediction/notebooks/../common/dataset.py:47: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  embeddings = torch.Tensor(embeddings)


In [11]:
# net = CNN()
# net = cait_S24_224()
net = cait_tmh()

# The function is indicating the performance of the model.
# During the training process this function should be minimized
criterion = nn.CrossEntropyLoss()

# The minimization is achieved through Stochastic Gradient Descent
optimizer = optim.Adagrad(net.parameters(), lr=0.01)

In [15]:
# single step
inputs, labels = iter(dataloader).__next__()

x = inputs.unsqueeze(1)

try:
    outputs = net(x)
except ValueError:
    import ipdb
    ipdb.post_mortem()

# print(f"{outputs} \t {labels}")
outputs.shape

# loss = criterion(outputs, labels.float())

tensor([[ 0.3019,  0.9502,  0.2833,  ..., -0.3675,  0.4257, -0.6928],
        [ 0.3020,  0.9502,  0.2832,  ..., -0.3674,  0.4258, -0.6928],
        [ 0.3019,  0.9503,  0.2834,  ..., -0.3674,  0.4257, -0.6929],
        [ 0.3019,  0.9503,  0.2832,  ..., -0.3674,  0.4259, -0.6929]],
       grad_fn=<AddmmBackward0>) 	 tensor([[0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]])


torch.Size([4, 1000])

In [14]:
net.blocks[0]

LayerScale_Block(
  (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
  (attn): Attention_talking_head(
    (qkv): Linear(in_features=1024, out_features=3072, bias=True)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=1024, out_features=1024, bias=True)
    (proj_l): Linear(in_features=8, out_features=8, bias=True)
    (proj_w): Linear(in_features=8, out_features=8, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (drop_path): Identity()
  (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
  (mlp): Mlp(
    (fc1): Linear(in_features=1024, out_features=4096, bias=True)
    (act): GELU()
    (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
)

In [30]:
for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        """
        shape of inputs: torch.Size([4, 3, 32, 32])
            Batchsize: 4
            Channels: 3 (Red, Green, Blue)
            Image size: 32 x 32

        labels: tensor([9, 3, 0, 3])
            9: class of image 0 in batch
            3: class of image 1 in batch
            ...
        """
        inputs, labels = data

        """ zero the parameter gradients after every batch
        This is necessary because the gradients (directions of how the weigths and biases
        will be updated) are accumulated in each backward pass.
        https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        """
        optimizer.zero_grad()  # SGD

        # forward + backward + optimize
        # shape outputs: torch.Size([4, 10])
        # for every image a prediction
        # print(f"{inputs}")
        outputs = net(inputs.unsqueeze(1))

        #print(f"{outputs} \t {labels}")

        # the first iteration CrossEntropy: tensor(2.3100, grad_fn=<NllLossBackward0>)

        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # running loss after 3 iterations: 6.894119024276733
        # Why is the loss added?
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {(running_loss / 99):.3f}')
            running_loss = 0.0

print('Finished Training')

[1, 100] loss: 1.187
[1, 200] loss: 1.053
[1, 300] loss: 1.056
[1, 400] loss: 1.092
[1, 500] loss: 1.132
[1, 600] loss: 1.108
[1, 700] loss: 1.023
[1, 800] loss: 1.001
[2, 100] loss: 0.872
[2, 200] loss: 0.716
[2, 300] loss: 0.600
[2, 400] loss: 0.521
[2, 500] loss: 0.479
[2, 600] loss: 0.416
[2, 700] loss: 0.350
[2, 800] loss: 0.344
[3, 100] loss: 0.275
[3, 200] loss: 0.241
[3, 300] loss: 0.245
[3, 400] loss: 0.231
[3, 500] loss: 0.238
[3, 600] loss: 0.217
[3, 700] loss: 0.218
[3, 800] loss: 0.222
[4, 100] loss: 0.171
[4, 200] loss: 0.163
[4, 300] loss: 0.178
[4, 400] loss: 0.171
[4, 500] loss: 0.179
[4, 600] loss: 0.157
[4, 700] loss: 0.177
[4, 800] loss: 0.179
Finished Training


### Evaluate the model on the test data
This could be done with TorchMetrics but we will do this manually here

In [31]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images.unsqueeze(1))

        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(labels.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(test_labels)} test embeddings: {(100 * correct / total):.2f}')

Accuracy of the network on the 1392 test embeddings: 96.05


In [32]:
# prepare to count predictions for each class
classes = list(label_mappings.keys())
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = net(images.unsqueeze(1))
        _, predictions = torch.max(outputs, 1)
        _, labels = torch.max(labels, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                         accuracy))

"""
Accuracy for class G_SP  is: 93.2 %
Accuracy for class G     is: 99.4 %
Accuracy for class SP_TM is: 87.6 %
Accuracy for class TM    is: 93.5 %
"""

Accuracy for class G_SP  is: 93.2 %
Accuracy for class G     is: 99.4 %
Accuracy for class SP_TM is: 87.6 %
Accuracy for class TM    is: 93.5 %


'\nAccuracy for class G_SP  is: 0.0 %\nAccuracy for class G     is: 100.0 %\nAccuracy for class SP_TM is: 0.0 %\nAccuracy for class TM    is: 0.0 %\n'

Create hashsum